In [1]:
import cfbd
import numpy as np
import pandas as pd
import pickle
from sklearn import tree
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


In [2]:
df = pd.read_csv('../cfb_imputed.csv')
test = pd.read_csv('../a_CFB_simple/df22.csv')
week4 = pd.read_csv('../a_CFB_simple/c.csv')
predictionsDB = pd.read_csv('../a_CFB_simple/d.csv')

In [7]:
# df = df.drop('Unnamed: 0',axis=1)
df.isnull().sum().max()

0

In [8]:
df.tail()

,id,year,week,neutral_site,home_team,home_conference,home_points,home_elo,away_team,away_conference,...,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,margin
4739,401282154,2021,14,True,Alabama,SEC,41,2173,Georgia,SEC,...,1.088737,0.487179,0.148148,1.999448,1.495395,0.471429,0.230769,26.427636,6.5,-17
4740,401331163,2021,14,True,Wake Forest,ACC,21,1738,Pittsburgh,ACC,...,1.195964,0.458333,0.333333,6.220499,1.231439,0.376471,0.176471,-3.280177,3.5,24
4741,401331447,2021,14,True,Iowa,Big Ten,3,1767,Michigan,Big Ten,...,0.952849,0.489796,0.120000,7.824925,1.180175,0.257143,0.096774,-6.631342,12.0,39
4742,401387250,2021,14,False,California,Pac-12,24,1499,USC,Pac-12,...,1.245379,0.407692,0.098361,11.238803,0.936044,0.425287,0.200000,10.075825,-4.5,-10
4743,401301056,2021,15,False,Army,FBS Independents,13,1596,Navy,American Athletic,...,1.562338,0.381818,0.187500,15.917426,1.169774,0.270833,0.187500,-5.384225,7.0,4


In [9]:
y = []
for i in range(len(df)):
    if df.spread[i] < 0:
        if df.margin[i] > df.spread[i]:
            y.append(0)
        else:
            y.append(1)
    else:
        if df.margin[i] < df.spread[i]:
            y.append(1)
        else:
            y.append(0)
            
df["home_team_cover_spread"] = y
df.head()

,id,year,week,neutral_site,home_team,home_conference,home_points,home_elo,away_team,away_conference,...,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,margin,home_team_cover_spread
0,400763593,2015,1,False,UCF,American Athletic,14,1626,Florida International,Conference USA,...,0.362319,0.233333,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,1,0
1,400603840,2015,1,True,South Carolina,SEC,17,1646,North Carolina,ACC,...,0.461538,0.142857,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,-4,1
2,400763399,2015,1,False,Central Michigan,Mid-American,13,1417,Oklahoma State,Big 12,...,0.426471,0.181818,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,11,1
3,400603839,2015,1,False,Vanderbilt,SEC,12,1365,Western Kentucky,Conference USA,...,0.396226,0.380952,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,2,0
4,400756883,2015,1,False,Utah,Pac-12,24,1603,Michigan,Big Ten,...,0.452055,0.241379,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,-7,1


In [10]:
excluded = ['id','week','home_team','away_team','margin', 'home_points', 'away_points', 'home_team_cover_spread','attendance']
cat_features = ['home_conference','away_conference','neutral_site']
cont_features = [c for c in df.columns.to_list() if c not in cat_features and c not in excluded]

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
norm = df[cont_features]
norm_train = norm.query("year != 2021")
norm_test = norm.query("year == 2021")
norm_train = norm_train.drop('year',axis=1)
norm_test = norm_test.drop('year', axis=1)
normed_train = scaler.fit_transform(norm_train)
normed_test = scaler.transform(norm_test)
print(normed_train.shape, '\n', normed_test.shape)
print(norm_test.columns)

(4014, 60) 
 (730, 60)
Index(['home_elo', 'away_elo', 'home_sp_plus', 'home_second_order_w',
       'home_sos', 'home_special_teams_rating', 'home_d_r_explosivenenss',
       'home_d_r_success', 'home_d_havoc_db', 'home_d_havoc_f7',
       'home_d_havoc_total', 'home_d_standardDowns', 'home_d_passing',
       'home_d_rushing', 'home_d_passingDowns', 'home_d_ranking',
       'home_o_r_explosiveness', 'home_o_r_success', 'home_o_standardDowns',
       'home_o_passing', 'home_o_rushing', 'home_o_runrate', 'home_o_pace',
       'home_o_passingDowns', 'home_o_ranking', 'away_sp_plus',
       'away_second_order_w', 'away_sos', 'away_special_teams_rating',
       'away_d_r_explosivenenss', 'away_d_r_success', 'away_d_havoc_db',
       'away_d_havoc_f7', 'away_d_havoc_total', 'away_d_standardDowns',
       'away_d_passing', 'away_d_rushing', 'away_d_passingDowns',
       'away_d_ranking', 'away_o_r_explosiveness', 'away_o_r_success',
       'away_o_standardDowns', 'away_o_passing', 'away_o_rus

In [12]:
cont_test = pd.DataFrame(normed_test, columns=norm_test.columns)
cont_train = pd.DataFrame(normed_train, columns=norm_train.columns)

In [13]:
conts = pd.concat([cont_train,cont_test],axis=0)
conts

,home_elo,away_elo,home_sp_plus,home_second_order_w,home_sos,home_special_teams_rating,home_d_r_explosivenenss,home_d_r_success,home_d_havoc_db,home_d_havoc_f7,...,drives,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread
0,0.570144,0.368390,0.269394,0.057971,0.744681,0.765625,0.312500,0.201872,0.333333,0.304348,...,0.4375,0.388337,0.320731,0.414815,0.548258,0.336639,0.476062,0.777778,0.375059,0.358140
1,0.582134,0.461081,0.535966,0.282609,0.297872,0.609375,0.579167,0.165775,0.115385,0.478261,...,0.3125,0.267919,0.426421,0.253968,0.453420,0.246537,0.508663,0.283688,0.429014,0.483721
2,0.444844,0.514557,0.444288,0.507246,0.787234,0.125000,0.520833,0.312834,0.435897,0.318841,...,0.4375,0.318746,0.389066,0.323232,0.580727,0.205233,0.492695,0.555556,0.419397,0.706977
3,0.413669,0.487225,0.465444,0.282609,0.391489,0.468750,0.429167,0.494652,0.705128,0.514493,...,0.5000,0.219352,0.356850,0.677249,0.414961,0.212513,0.461988,0.487805,0.287224,0.353488
4,0.556355,0.506239,0.643159,0.492754,0.459574,0.796875,0.654167,0.454545,0.538462,0.768116,...,0.3750,0.288185,0.416319,0.429119,0.343901,0.229258,0.457086,0.416667,0.408007,0.488372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,0.898082,0.884730,0.854725,0.943528,0.210418,0.489149,0.562859,0.887193,0.706012,0.717141,...,0.5000,0.256433,0.453735,0.263374,0.382001,0.386473,0.580137,0.384615,0.619690,0.576744
726,0.637290,0.618538,0.630465,0.419302,0.448743,0.593192,0.500473,0.366553,0.446387,0.499671,...,0.6875,0.302220,0.423007,0.592593,0.432506,0.285225,0.428511,0.294118,0.291695,0.548837
727,0.654676,0.764706,0.648801,0.591594,0.513191,0.605469,0.725750,0.388877,0.391282,0.533623,...,0.4375,0.198408,0.456522,0.213333,0.451703,0.265561,0.237973,0.161290,0.254696,0.627907
728,0.494005,0.473559,0.535966,0.435413,0.500463,0.505580,0.643388,0.298593,0.420847,0.335854,...,0.5625,0.323320,0.369064,0.174863,0.492551,0.171918,0.506460,0.333333,0.439154,0.474419


In [14]:
# cats = df[cat_features]
# cats = cats.apply(preprocessing.LabelEncoder().fit_transform)
# cats
onehots = df[cat_features]
onehots.columns

Index(['home_conference', 'away_conference', 'neutral_site'], dtype='object')

In [15]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder(sparse=False)
onehots = df[cat_features]
con = pd.DataFrame(oh.fit_transform(df[['home_conference']]))
con.columns = oh.get_feature_names(['home_conference'])
con

C:\Users\magamp\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,home_conference_ACC,home_conference_American Athletic,home_conference_Big 12,home_conference_Big Ten,home_conference_Conference USA,home_conference_FBS Independents,home_conference_Mid-American,home_conference_Mountain West,home_conference_Pac-12,home_conference_SEC,home_conference_Sun Belt
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4740,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4741,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
oh = OneHotEncoder(sparse=False)
onehots = df[cat_features]
acon = pd.DataFrame(oh.fit_transform(df[['away_conference']]))
acon.columns = oh.get_feature_names(['away_conference'])
acon

C:\Users\magamp\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,away_conference_ACC,away_conference_American Athletic,away_conference_Big 12,away_conference_Big Ten,away_conference_Conference USA,away_conference_FBS Independents,away_conference_Mid-American,away_conference_Mountain West,away_conference_Pac-12,away_conference_SEC,away_conference_Sun Belt
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4740,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4741,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
neut = pd.DataFrame(oh.fit_transform(df[['neutral_site']])).reset_index(drop=True)
neut.columns = oh.get_feature_names(['neutral_site'])
neut

C:\Users\magamp\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,neutral_site_False,neutral_site_True
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
4739,0.0,1.0
4740,0.0,1.0
4741,0.0,1.0
4742,1.0,0.0


In [18]:
leftovers = df[excluded]
leftovers['year'] = df['year']
leftovers = leftovers.reset_index( drop=True)
leftovers

C:\Users\magamp\AppData\Local\Temp\ipykernel_5564\214285720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leftovers['year'] = df['year']


,id,week,home_team,away_team,margin,home_points,away_points,home_team_cover_spread,attendance,year
0,400763593,1,UCF,Florida International,1,14,15,0,39184.0,2015
1,400603840,1,South Carolina,North Carolina,-4,17,13,1,51664.0,2015
2,400763399,1,Central Michigan,Oklahoma State,11,13,24,1,19717.0,2015
3,400603839,1,Vanderbilt,Western Kentucky,2,12,14,0,30307.0,2015
4,400756883,1,Utah,Michigan,-7,24,17,1,47825.0,2015
...,...,...,...,...,...,...,...,...,...,...
4739,401282154,14,Alabama,Georgia,-17,41,24,1,78030.0,2021
4740,401331163,14,Wake Forest,Pittsburgh,24,21,45,0,57856.0,2021
4741,401331447,14,Iowa,Michigan,39,3,42,0,67183.0,2021
4742,401387250,14,California,USC,-10,24,14,1,42076.0,2021


In [19]:
print(leftovers.columns,acon.columns,con.columns,conts.columns)

Index(['id', 'week', 'home_team', 'away_team', 'margin', 'home_points',
       'away_points', 'home_team_cover_spread', 'attendance', 'year'],
      dtype='object') Index(['away_conference_ACC', 'away_conference_American Athletic',
       'away_conference_Big 12', 'away_conference_Big Ten',
       'away_conference_Conference USA', 'away_conference_FBS Independents',
       'away_conference_Mid-American', 'away_conference_Mountain West',
       'away_conference_Pac-12', 'away_conference_SEC',
       'away_conference_Sun Belt'],
      dtype='object') Index(['home_conference_ACC', 'home_conference_American Athletic',
       'home_conference_Big 12', 'home_conference_Big Ten',
       'home_conference_Conference USA', 'home_conference_FBS Independents',
       'home_conference_Mid-American', 'home_conference_Mountain West',
       'home_conference_Pac-12', 'home_conference_SEC',
       'home_conference_Sun Belt'],
      dtype='object') Index(['home_elo', 'away_elo', 'home_sp_plus', 'home_se

In [20]:
df2 = pd.concat([ leftovers, con, acon, neut,conts.reset_index()], axis=1)
df2 = df2.drop(['attendance','id','week','home_team','away_team','margin', 'home_points', 'away_points'],axis=1)
df2

,home_team_cover_spread,year,home_conference_ACC,home_conference_American Athletic,home_conference_Big 12,home_conference_Big Ten,home_conference_Conference USA,home_conference_FBS Independents,home_conference_Mid-American,home_conference_Mountain West,...,drives,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread
0,0,2015,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4375,0.388337,0.320731,0.414815,0.548258,0.336639,0.476062,0.777778,0.375059,0.358140
1,1,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3125,0.267919,0.426421,0.253968,0.453420,0.246537,0.508663,0.283688,0.429014,0.483721
2,1,2015,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.4375,0.318746,0.389066,0.323232,0.580727,0.205233,0.492695,0.555556,0.419397,0.706977
3,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5000,0.219352,0.356850,0.677249,0.414961,0.212513,0.461988,0.487805,0.287224,0.353488
4,1,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3750,0.288185,0.416319,0.429119,0.343901,0.229258,0.457086,0.416667,0.408007,0.488372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4739,1,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5000,0.256433,0.453735,0.263374,0.382001,0.386473,0.580137,0.384615,0.619690,0.576744
4740,0,2021,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.6875,0.302220,0.423007,0.592593,0.432506,0.285225,0.428511,0.294118,0.291695,0.548837
4741,0,2021,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.4375,0.198408,0.456522,0.213333,0.451703,0.265561,0.237973,0.161290,0.254696,0.627907
4742,1,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5625,0.323320,0.369064,0.174863,0.492551,0.171918,0.506460,0.333333,0.439154,0.474419


In [55]:
df2.to_csv('NNData_87.csv')

In [21]:
test_df = df2.query("year == 2021")
train_df = df2.query("year != 2021")
test_df.head()

,home_team_cover_spread,year,home_conference_ACC,home_conference_American Athletic,home_conference_Big 12,home_conference_Big Ten,home_conference_Conference USA,home_conference_FBS Independents,home_conference_Mid-American,home_conference_Mountain West,...,drives,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread
4014,1,2021,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.4375,0.404618,0.384874,0.313725,0.630643,0.210366,0.542346,0.333333,0.373932,0.576744
4015,1,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.5625,0.272591,0.109408,0.380952,0.068636,0.265665,0.615390,0.350877,0.575121,0.260465
4016,1,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5625,0.250465,0.294653,0.654971,0.315272,0.352633,0.490648,0.277778,0.489781,0.353488
4017,0,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.4375,0.392344,0.379948,0.115942,0.617574,0.288884,0.221099,0.245098,0.174284,0.604651
4018,0,2021,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5000,0.353758,0.311706,0.888889,0.456927,0.313702,0.578795,0.144928,0.537773,0.455814


In [22]:
trainx = train_df.drop(['home_team_cover_spread', 'year'], axis=1)
testx = test_df.drop(['home_team_cover_spread', 'year'], axis=1)

In [23]:
trainx = trainx.drop('index', axis=1)

In [24]:
testx = testx.drop('index', axis=1)

In [25]:
# from sklearn.preprocessing import StandardScaler  
# scaler = StandardScaler()
# scaler.fit(trainx)
# x_train2 = scaler.transform(trainx)
# x_test2 = scaler.transform(testx)

In [26]:
trainy = train_df[["home_team_cover_spread"]]
testy = test_df[["home_team_cover_spread"]]

In [27]:
# nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,2), random_state=1)

In [28]:
# nn.fit(x_train2,trainy)

In [29]:
# nn.predict(testx)

In [30]:
def get_mlp_model(hiddenLayerOne=784, hiddenLayerTwo=256,
	dropout=0.2, learnRate=0.01):
	# initialize a sequential model and add layer to flatten the
	# input data
	model = Sequential()
	model.add(Flatten())
 	# add two stacks of FC => RELU => DROPOUT
	model.add(Dense(hiddenLayerOne, activation="relu",
		input_shape=(784,)))
	model.add(Dropout(dropout))
	model.add(Dense(hiddenLayerTwo, activation="relu"))
	model.add(Dropout(dropout))
	# add a softmax layer on top
	model.add(Dense(10, activation="softmax"))
	# compile the model
	model.compile(
		optimizer=Adam(learning_rate=learnRate),
		loss="sparse_categorical_crossentropy",
		metrics=["accuracy"])
	# return compiled model
	return model

In [31]:
import tensorflow as tf
tf.random.set_seed(42)

In [32]:
print("[INFO] initializing model...")
model = get_mlp_model()

[INFO] initializing model...


In [33]:
print("[INFO] training model...")
H = model.fit(x=trainx, y=trainy,
	validation_data=(testx, testy),
	batch_size=8,
	epochs=20)

[INFO] training model...
Epoch 1/20
502/502 [==============================] - 2s 3ms/step - loss: 0.7268 - accuracy: 0.5254 - val_loss: 0.7066 - val_accuracy: 0.4890
Epoch 2/20
502/502 [==============================] - 1s 2ms/step - loss: 0.6895 - accuracy: 0.5486 - val_loss: 0.6363 - val_accuracy: 0.6630
Epoch 3/20
502/502 [==============================] - 1s 2ms/step - loss: 0.6809 - accuracy: 0.5660 - val_loss: 0.7012 - val_accuracy: 0.4890
Epoch 4/20
502/502 [==============================] - 1s 2ms/step - loss: 0.6969 - accuracy: 0.5015 - val_loss: 0.6929 - val_accuracy: 0.5110
Epoch 5/20
502/502 [==============================] - 1s 2ms/step - loss: 0.6957 - accuracy: 0.5002 - val_loss: 0.6981 - val_accuracy: 0.4890
Epoch 6/20
502/502 [==============================] - 1s 2ms/step - loss: 0.6958 - accuracy: 0.4953 - val_loss: 0.6935 - val_accuracy: 0.4890
Epoch 7/20
502/502 [==============================] - 1s 2ms/step - loss: 0.6956 - accuracy: 0.5105 - val_loss: 0.6930 - va

OPTIMIZE HYPERPARAMETERS USING SKLEARN

In [34]:
# wrap our model into a scikit-learn compatible classifier
print("[INFO] initializing model...")
model = KerasClassifier(build_fn=get_mlp_model, verbose=0)

[INFO] initializing model...


C:\Users\magamp\AppData\Local\Temp\ipykernel_5564\1675106929.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=get_mlp_model, verbose=0)


In [35]:
# define a grid of the hyperparameter search space
hiddenLayerOne = [256, 512, 784]
hiddenLayerTwo = [128, 256, 512]
learnRate = [1e-2, 1e-3, 1e-4]
dropout = [0.3, 0.4, 0.5]
batchSize = [4, 8, 16, 32]
epochs = [10, 20, 30, 40]

In [36]:
# create a dictionary from the hyperparameter grid
grid = dict(
	hiddenLayerOne=hiddenLayerOne,
	learnRate=learnRate,
	hiddenLayerTwo=hiddenLayerTwo,
	dropout=dropout,
	batch_size=batchSize,
	epochs=epochs
)

In [37]:
# initialize a random search with a 5-fold cross-validation and then
# start the hyperparameter search process
print("[INFO] performing random search...")
searcher = RandomizedSearchCV(estimator=model, n_jobs=-1, cv=5,
	param_distributions=grid, scoring="accuracy")
searchResults = searcher.fit(trainx, trainy)

[INFO] performing random search...


In [38]:

# summarize grid search information
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("[INFO] best score is {:.2f} using {}".format(bestScore,
	bestParams))

[INFO] best score is 0.78 using {'learnRate': 0.0001, 'hiddenLayerTwo': 128, 'hiddenLayerOne': 512, 'epochs': 30, 'dropout': 0.3, 'batch_size': 8}


In [39]:
testx.head()

,home_conference_ACC,home_conference_American Athletic,home_conference_Big 12,home_conference_Big Ten,home_conference_Conference USA,home_conference_FBS Independents,home_conference_Mid-American,home_conference_Mountain West,home_conference_Pac-12,home_conference_SEC,...,drives,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread
4014,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4375,0.404618,0.384874,0.313725,0.630643,0.210366,0.542346,0.333333,0.373932,0.576744
4015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.5625,0.272591,0.109408,0.380952,0.068636,0.265665,0.615390,0.350877,0.575121,0.260465
4016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.5625,0.250465,0.294653,0.654971,0.315272,0.352633,0.490648,0.277778,0.489781,0.353488
4017,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.4375,0.392344,0.379948,0.115942,0.617574,0.288884,0.221099,0.245098,0.174284,0.604651
4018,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5000,0.353758,0.311706,0.888889,0.456927,0.313702,0.578795,0.144928,0.537773,0.455814


OPTIMIZE HYPERPARAMETERS USING ML ROSE

In [40]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np

Begin with random hill climb

In [292]:
# nn_model1 = mlrose.NeuralNetwork(hidden_nodes = [2], activation = 'relu', algorithm = 'random_hill_climb', max_iters = 1000, bias = True, is_classifier = True, learning_rate = 0.0001,
# early_stopping = True, clip_max = 5, max_attempts = 100, random_state = 3)

In [293]:
# nn_model1.fit(trainx,trainy)

NeuralNetwork(clip_max=5, early_stopping=True, hidden_nodes=[2],
              learning_rate=0.0001, max_iters=1000)

In [294]:
# from sklearn.metrics import accuracy_score
# # Predict labels for train set and assess accuracy
# y_train_pred = nn_model1.predict(trainx)

# y_train_accuracy = accuracy_score(trainy, y_train_pred)

# print('Training accuracy: ', y_train_accuracy)

# # Predict labels for test set and assess accuracy
# y_test_pred = nn_model1.predict(testx)

# y_test_accuracy = accuracy_score(testy, y_test_pred)

# print('Test accuracy: ', y_test_accuracy)

Training accuracy:  0.4683607374190334
Test accuracy:  0.4095890410958904


Lets Try Gradient Descent

In [41]:
# nn_model2 = mlrose.NeuralNetwork(hidden_nodes = [100], activation = 'relu', algorithm = 'gradient_descent', max_iters = 1000, bias = True, is_classifier = True, learning_rate = 0.0001,
# early_stopping = True, clip_max = 5, max_attempts = 1000, random_state = 3)

In [42]:
# nn_model2.fit(trainx,trainy)

NeuralNetwork(clip_max=5, early_stopping=True, hidden_nodes=[100],
              learning_rate=0.0001, max_iters=1000)

In [43]:
# # Predict labels for train set and assess accuracy
# y_train_pred = nn_model2.predict(trainx)

# y_train_accuracy = accuracy_score(trainy, y_train_pred)

# print('Training accuracy: ', y_train_accuracy)

# # Predict labels for test set and assess accuracy
# y_test_pred = nn_model2.predict(testx)

# y_test_accuracy = accuracy_score(testy, y_test_pred)

# print('Test accuracy: ', y_test_accuracy)

Training accuracy:  0.7678126557050324
Test accuracy:  0.7246575342465753


## Best NN Below
### Could still optimize number of hidden layers

In [50]:
nn_model4 = mlrose.NeuralNetwork(hidden_nodes = [6,6], activation = 'relu', algorithm = 'gradient_descent', max_iters = 5000, bias = True, is_classifier = True, learning_rate = 0.0001,
early_stopping = True, clip_max = 5, max_attempts = 1000, random_state = 3)

In [52]:
nn_model4.fit(trainx,trainy)

NeuralNetwork(clip_max=5, early_stopping=True, hidden_nodes=[6, 6],
              learning_rate=0.0001, max_iters=5000)

In [53]:
# Predict labels for train set and assess accuracy
y_train_pred = nn_model4.predict(trainx)

y_train_accuracy = accuracy_score(trainy, y_train_pred)

print('Training accuracy: ', y_train_accuracy)

# Predict labels for test set and assess accuracy
y_test_pred = nn_model4.predict(testx)

y_test_accuracy = accuracy_score(testy, y_test_pred)

print('Test accuracy: ', y_test_accuracy)

Training accuracy:  0.8183856502242153
Test accuracy:  0.8013698630136986


In [56]:
# nn_model4.fitted_weights